In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
from libra_py import DFTB_methods
from libra_py import units


/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

In [2]:
def do_step(i, EXE, norbs):
    
    # normal size of the active space (AO basis)
    act_sp = list(range(0,norbs))
    
    # Make an input file for SP calculations 
    DFTB_methods.xyz_traj2gen_sp("md.xyz", "x1.gen", i, "C")

    # Run SCF calculations and generate the charge density for a converged calculations
    # The file x1.gen is used as a geometry
    os.system("cp dftb_in_ham1.hsd dftb_in.hsd")
    os.system( "%s" % EXE )

    # Just generate the Hamiltonian corresponding to the converged density matrix
    os.system("cp dftb_in_ham2.hsd dftb_in.hsd")
    os.system( "%s" % EXE )

    # Get the Hamiltonian    
    Hi = DFTB_methods.get_dftb_matrices("hamsqr1.dat", act_sp, act_sp)    
    Si = DFTB_methods.get_dftb_matrices("oversqr.dat", act_sp, act_sp) 
    Ei = CMATRIX(norbs, norbs)
    MOi = CMATRIX(norbs, norbs)
    solve_eigen(Hi[0], Si[0], Ei, MOi, 0)  # gamma-point only at time i
    
    return Ei, MOi


def do_ovlp(i, EXE, norbs):
        
    # Make an input file for the overlap calculations 
    DFTB_methods.xyz_traj2gen_ovlp("md.xyz", "x2.gen", i, i+1, "C")
    
    # Run SCF calculations and generate the charge density for a converged calculations
    # The file x2.gen is used as a geometry
    os.system("cp dftb_in_overlaps.hsd dftb_in.hsd")
    os.system( "%s" % EXE )

    # Get the Hamiltonian    
    act_sp = list(range(0,2*norbs))
    Sbig = DFTB_methods.get_dftb_matrices("oversqr.dat", act_sp, act_sp)    
    
    act_sp1 = list(range(0,norbs))
    act_sp2 = list(range(norbs,2*norbs))
    S = CMATRIX(norbs, norbs)
    pop_submatrix(Sbig[0], S, act_sp1, act_sp2)
    
    return S
    
    
    
def run_step2(nsteps, EXE, norbs, dt):

    U_curr = CMATRIX(norbs, norbs)
    U_next = CMATRIX(norbs, norbs)
    E_curr = CMATRIX(norbs, norbs)
    E_next = CMATRIX(norbs, norbs)
        
    act_sp1 = list(range(0,norbs))  # normal size of the active space (AO basis)
    
    E_curr, U_curr = do_step(0, EXE, norbs)
    
    for i in range(1,nsteps-1):
        E_next, U_next = do_step(i, EXE, norbs)
        S = do_ovlp(i, EXE, norbs)
        
        TDM = U_curr.H() * S * U_next
        Hvib = 0.5*(E_curr + E_next) - (0.5j/dt) * ( TDM - TDM.H() )
        
        
        Hvib.real().show_matrix("res/hvib_%i_re" % (i) )
        Hvib.imag().show_matrix("res/hvib_%i_im" % (i) )
        
        # Current becomes the old 
        E_curr = CMATRIX(E_next)
        U_curr = CMATRIX(U_next)
        

In [3]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/dftb/dftbplus-18.2.x86_64-linux/bin/dftb+"
nsteps = 5
norbs = 87
dt = 1.0 * units.fs2au


os.system("mkdir res")
run_step2(nsteps, EXE, norbs, dt)